## AEX - Arbitrum Ecosystem Index - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  1e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = 'arbitrum-ecosystem'

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results,liquidity = methodology_class.main(
    single_chain='arbitrum-one',
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['chainlink','wrapped-bitcoin','uniswap','lido-dao']
    )
liquidity

0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x95ab45875cffdba1e5f451b950bc2e42c0053f39 arbitrum-one
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861 arbitrum-one
0xcaa38bcc8fb3077975bbe217acfaa449e6596a84 arbitrum-one
0xa68ec98d7ca870cf1dd0b00ebbb7c4bf60a8e74d arbitrum-one
0xd58d345fd9c82262e087d2d0607624b410d88242 arbitrum-one
0xf0cb2dc0db5e6c66b9a70ac27b06b878da017028 arbitrum-one
0xf3c091ed43de9c270593445163a41a876a0bb3dd arbitrum-one
0x560363bda52bc6a44ca6c8c9b4a5fadbda32fa60 arbitrum-one
0xfb9e5d956d889d91a82737b9bfcdac1dce3e1449 arbitrum-one
0x319f865b287fcc10b30d8ce6144e8b6d1b476999 arbitrum-one
0xbc011a12da28e8f0f528d9ee5e7039e22f91cf18 arbitrum-one
Dropping rocket-pool-eth because a price feed is unavailable
Dropping bitcoin-avalanche-bridged-btc-b because a price feed is unavailable
Dropping radiant-capital because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
rocket-pool-eth,0.000393,3.925300e-04,-0.000512,arbitrum-one
pendle,0.764489,7.604185e-01,-0.005324,arbitrum-one
magic,1.087191,1.081371e+00,-0.005353,arbitrum-one
gmx,0.018978,1.880928e-02,-0.008865,arbitrum-one
bitcoin-avalanche-bridged-btc-b,0.000023,2.272000e-05,-0.009590,arbitrum-one
radiant-capital,3.445501,3.405981e+00,-0.011470,arbitrum-one
woo-network,4.126448,4.068045e+00,-0.014153,arbitrum-one
gains-network,0.251036,2.460861e-01,-0.019719,arbitrum-one
joe,1.884481,1.837189e+00,-0.025096,arbitrum-one


## Show Results

In [8]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
gmx,GMX,490224880,0.300000,77,0xfc5a1a6eb076a2c7ad06ed22c90d7e710e35ad0a,arbitrum-one
woo-network,WOO Network,429374686,0.271505,70,0xcafcd85d8ca7ad1e1c6f82f651fa15e33aefd07b,arbitrum-one
magic,Magic,235033858,0.148618,38,0x539bde0d7dbd336b79148aa742883198bbf60342,arbitrum-one
joe,JOE,182073069,0.115130,29,0x371c7ec6d8039ff7933a2aa28eb827ffe1f52f07,arbitrum-one
gains-network,Gains Network,133960056,0.084706,21,0x18c11fd286c5ec11c3b683caa813b77f5163a122,arbitrum-one
pendle,Pendle,126581707,0.080041,20,0x0c880f6761f1af8d9aa9c466984b80dab9a8c9e8,arbitrum-one
